FCC Pythia + Delphes + Heppy Analysis
==========================================================

-   [Overview](#overview)
-   [Generate and Simulate Events](#generate-events)
-   [Analyze Events](#analyze-events)
-   [Plot events](#plot-events)
-   [Homework exercise](#homework-exercise)

Overview
---------------------

This tutorial will teach you how to:

-   **generate** signal and background samples with **Pythia8** within FCCSW
-   run a fast parametric **detector simulation** with **Delphes** within FCCSW
-   apply an **event selection** on those samples with **Heppy**
-   produce **flat ntuples** with observables of interest with Heppy
-   produce plots

**This tutorial has been tested on bash shells. It is not guaranteed to work on other shells.**

Part I: Generate and simulate Events with FCCSW
----------------------------------------------------

First, make sure your **setup of the FCC software** is working correctly. A quick check is that the executable `fccrun`, which allows you to run jobs in the Gaudi framework is available on the command line:

In [1]:
%%bash
which fccrun

/usr/local/scripts//fccrun


If the above command fails without printing a path like `/cvmfs/fcc.cern.ch/sw/releases/fccsw/0.10/x86_64-centos7-gcc62-opt/scripts/fccrun` check [the relevant documentation page to setup FCC software](https://github.com/HEP-FCC/fcc-tutorials/blob/master/FccSoftwareGettingStarted.md).

For this tutorial we will consider the following **physics processes**:

-   p p -> H -> 4 l
-   p p -> Z/gamma Z/gamma -> 4 l

Pythia can be configured to hadronize previously generated hard scattering in the form of Les Houches event files (.lhe),
or generate the hard process itself and then run the parton shower and hadronization.
**In either case, the FCCSW takes as input a Pythia8 configuration file** (.cmd), and does not need to know which approach was used. 

For this tutorial, we are going to run Pythia8 on previously produced LHE files (with [MG5_aMCatNLO](https://launchpad.net/mg5amcnlo)).
Additional Pythia8 configurations are present in `Generation/data`. 

The following commands will run Pythia8 and Delphes and produce the relevant signal and background samples:

In [3]:
%%bash
 fccrun PythiaDelphes_config.py --Filename Pythia_pp_h_4l.cmd --filename pp_h_4l.root -n 1000
 fccrun PythiaDelphes_config.py --Filename Pythia_pp_zgzg_4l.cmd --filename pp_zgzg_4l.root -n 1000

 -->  GenAlg  -->  Converter  -->  DelphesSimulation  -->  out  

ApplicationMgr    SUCCESS 
                                                   Welcome to ApplicationMgr (GaudiCoreSvc v32r0)
                                          running on vali-XPS-13-9360 on Thu Aug 15 10:43:12 2019
ApplicationMgr       INFO Application Manager Configured successfully
GenAlg.PileUpTool    INFO Current number of pileup events:  'numPileUpEvents':0
RndmGenSvc.Engine    INFO Generator engine type:CLHEP::RanluxEngine
RndmGenSvc.Engine    INFO Current Seed:1234567 Luxury:3
RndmGenSvc           INFO Using Random engine:HepRndm::Engine<CLHEP::RanluxEngine>
GenAlg.VertexSm...   INFO Smearing of interaction point with flat distribution  in x, y and z 
GenAlg.VertexSm...   INFO  applying TOF of interaction with positive beam direction
GenAlg.VertexSm...   INFO  with 0 mm <= x <= 0 mm, 0 mm <= y <= 0 mm and 0 mm <= z <= 0 mm.

 *--------------------------------------------------------------------------------

Error in <TTree::Bronch>: Cannot find dictionary for class: HepMC::GenEvent
Warning in <TFile::Init>: no StreamerInfo found in /usr/local/lib/libClassesDict_rdict.pcm therefore preventing schema evolution when reading this file.
Warning in <TFile::Init>: no StreamerInfo found in /usr/local/lib/libFastJetDict_rdict.pcm therefore preventing schema evolution when reading this file.
Warning in <TFile::Init>: no StreamerInfo found in /usr/local/lib/libModulesDict_rdict.pcm therefore preventing schema evolution when reading this file.
Error in <TCling::RegisterModule>: cannot find dictionary module libPythia8Dict_rdict.pcm
Warning in <TFile::Init>: no StreamerInfo found in /usr/local/lib/libExRootAnalysisDict_rdict.pcm therefore preventing schema evolution when reading this file.
Error in <TTree::Bronch>: Cannot find dictionary for class: HepMC::GenEvent
Warning in <TFile::Init>: no StreamerInfo found in /usr/local/lib/libClassesDict_rdict.pcm therefore preventing schema evolution when readi

The `--inputfile` , `--outputfile` and `--nevents` options simply overwrite parameters that are defined in the main
configuration `Sim/SimDelphesInterface/options/PythiaDelphes_config.py`. For a complete discussion on the structure of configuration file, see [this page](https://github.com/HEP-FCC/FCCSW/blob/master/Sim/SimDelphesInterface/doc/FccPythiaDelphes.md).
Aside from I/O and number of events (which can be specified through command line),
for most use cases as a user you won't need to apply any change to the configuration file.

In addition to the **sequence of modules** to be executed, and which **output collections** to be stored in the output tree, the following
parameters can be specified via the configuration file:

-   `nEvents` --&gt; number
    of events to be simulated
-   `pythiaConfFile` --&gt;
    Pythia8 configuration file
-   `delphescard` --&gt;
    Delphes detector card
-   `out.filename` --&gt;
    name of output file

In this example, Delphes is run on the fly with the baseline FCC-hh detector configuration.
Other detector cards can be found in the `$DELPHES_DIR/cards` directory, such as a ATLAS, CMS or ILD detector configurations:
`delphes_card_ATLAS.tcl`, `delphes_card_CMS.tcl` and `delphes_card_ILD.tcl`. Many of the questions you might have on Delphes Fast Simulation are probably answered
[here](https://cp3.irmp.ucl.ac.be/projects/delphes/wiki/WorkBook).

Now overwrite the samples you just produced, with larger samples (10k events) that have been previously produced and stored in eos.

The output is a ROOT file containing a tree in the FCC [Event Data Model structure](https://github.com/HEP-FCC/fcc-edm). It is browsable with ROOT:

In [4]:
%%bash 
rootls -t pp_h_4l.root

TTree  Aug 15 10:43 2019 events    "Events tree"
  Cluster INCLUSIVE ranges:
  The total number of clusters is 0
TTree  Aug 15 10:43 2019 metadata  "Metadata tree"
  gaudiConfigOptions  "gaudiConfigOptions"  3502
  CollectionIDs       "CollectionIDs"       0
    m_collectionIDs  "m_collectionIDs"  108
    m_names          "m_names"          100
  Cluster INCLUSIVE ranges:
   - # 0: [0, 0]
  The total number of clusters is 1


Plotting some basic quantities directly on this output is possible, although not very handy:

<img src="./images/FccFullAnalysis/ele1_pt.png" width="400">

[]()Part II: Analyze the output with Heppy
------------------------------------------

[Heppy](https://github.com/HEP-FCC/heppy) is a python framework suitable for analyzing the FCCSW output.

First install HEPPY:


Understand the configuration file for this **H->4l analysis**: `heppy/test/analysis_pp_hTo4l_simple_cfg.py`
This is where **filters** on input collections and **event selection** are defined.
The sequence is divided in two parts, a gen level analysis, and a reco level.

-   The **gen level analysis** simply filters interesting leptons (`gen_leptons`) and stores pT, eta in in flat tree (`gen_tree`).

Have a look at the corresponding code in `heppy/analyzers/examples/hzz4l/HTo4lGenTreeProducer.py`.

-   The **reco level analysis** first **selects isolated leptons** (`selected_muons`, `selected_electrons`), merges them into a single collection (`selected_leptons`),
**builds Z candidates** (`zeds`) and finally **builds higgs candidates**  (`higgses`). After that an **event selection** is applied (`selection`).

Open `heppy/analyzers/examples/hzz4l/selection.py` and understand the event selection.

Finally another flat tree is produced `HTo4lTreeProducer`.
This tree contains contains all relevant information for the two reconstructed Z bosons, the Higgs, and the four associated leptons.
For comparison, also the MC level counterparts of the reconstructed quantities are stored.

To summarize, when designing a new analysis, you will have to define:

- a configuration file containing the analysis sequence
- an event selection
- one or several tree producer(s) where the variables to be stored in the output tree(s) are specified
- optionally, new modules that are specific to your analysis (e.g. `LeptonicZedBuilder` here)

Now run HEPPY:

The runs create two output directories `pp_h_4l` and `pp_zgzg_4l`, with various subdirectories. The breakdown of
efficiencies by cut can be found in:

`example/heppy.analyzers.examples.hzz4l.selection.Selection_cuts/cut_flow.txt`

The **gen-level** and **reco-level** output trees can be found here:

`example/heppy.analyzers.examples.hzz4l.HTo4lGenTreeProducer.HTo4lGenTreeProducer_1/tree.root`
`example/heppy.analyzers.examples.hzz4l.HTo4lTreeProducer.HTo4lTreeProducer_1/tree.root`

[]()Part III: Produce plots
-----------------------

Download the python code:

Produce Gen-level plots:

<img src="./images/FccFullAnalysis/lep1vsPt_pt.png" width="400">
<img src="./images/FccFullAnalysis/lep1vsEta_eta.png" width="400">

Produce Reco-level plots:

Appreciate the signal yield for **25 fb-1 of data**. Compare with [ATLAS results](https://arxiv.org/pdf/1408.5191v3.pdf) at 7,8 TeV .

<img src="./images/FccFullAnalysis/higgs_m.png" width="400">
<img src="./images/FccFullAnalysis/m4l_80_170_allYear_125.png" width="400">

[]()Part IV: Homework
---------------------

As an exercise you can **re-run the full analysis with the CMS detector configuration** .Additional Delphes card can
be found in ```$DELPHES_DIR/cards```). 

Hint: The input Delphes card can be specified in the command line argument: 

You should obtain a plot similar to the following:

<img src="./images/FccFullAnalysis/higgs_m_cms.png" width="400">

By comparing the CMS and FCC Delphes cards, try to explain:

-  the **event yields difference between 7/8 TeV and 100 TeV** (by comparing the official ATLAS plot with the CMS configuration you just ran at 100 TeV)
-  the **event yields difference at 100 TeV** between the CMS and FCC detector configurations
-  the **difference in the width of Z and H peaks** between the CMS and FCC detector configurations

[]() Other documentation
------------------------

-   [Submit FCCSW jobs using the condor queue](https://github.com/HEP-FCC/fcc-tutorials/blob/master/FccCondor.md)
-   [Submit Heppy jobs using the batch queue](https://github.com/HEP-FCC/heppy/blob/master/doc/Heppy_-_Parallel_Processing.md)
-   [FCCSW webpage](http://fccsw.web.cern.ch/fccsw/index.html)
-   [Pythia8 manual](http://home.thep.lu.se/~torbjorn/pythia81html/Welcome.html)
-   [Delphes website](https://cp3.irmp.ucl.ac.be/projects/delphes)

------------------------------------------------------------------------





























